In [1]:
import dask
import dask.dataframe as dd

In [2]:
from dask.distributed import LocalCluster
cluster = LocalCluster()
client = cluster.get_client()

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import LabelEncoder
from sklearn.multioutput import MultiOutputClassifier

# from xgboost import XGBClassifier

import hvplot.dask

In [4]:
# train = pd.read_csv("data/midas_imp_1.csv")
# test = pd.read_csv("data/midas_imp_test_1.csv")

# dt1 = pd.to_datetime(dd.read_csv("data/training_set_VU_DM.csv").date_time, format= "%Y-%m-%d %H:%M:%S")
# dt2 = pd.to_datetime(dd.read_csv("data/test_set_VU_DM.csv").date_time, format= "%Y-%m-%d %H:%M:%S")


In [5]:
# train.insert(1, "date_time", dt1)
# test.insert(1, "date_time", dt2)

In [9]:
# train.to_csv("data/train_full.csv", index = False)
# test.to_csv("data/test_full.csv", index = False)

In [10]:
# use only 10% of the data, so train_small is 10% of train
# train_small = train.head(int(len(train)*0.1))
# test_small = test.head(int(len(test)*0.1))

# train_small.to_csv("data/train_small.csv", index = False)
# test_small.to_csv("data/test_small.csv", index = False)

In [11]:
# train = dd.read_csv("data/train_full.csv")
# test = dd.read_csv("data/test_full.csv")

# list of feautures that consist of more than 90% of NULL
dump = ["visitor_hist_starrating", "visitor_hist_adr_usd", "srch_query_affinity_score", 
        "comp1_rate", "comp1_inv", "comp1_rate_percent_diff", 
        "comp4_rate", "comp4_inv", "comp4_rate_percent_diff",
         "comp6_rate", "comp6_inv", "comp6_rate_percent_diff",
         "comp7_rate", "comp7_inv", "comp7_rate_percent_diff",
         ]

# remove hist star rating and visitor_hist_adr_usd cause 95% was null and tus ant be really used 
train_small = pd.read_csv("data/train_small.csv").drop(["visitor_hist_starrating", "visitor_hist_adr_usd", "srch_query_affinity_score", comp1_rate], axis=1)
train_small["date_time"] = pd.to_datetime(train_small["date_time"])

train_small["year"] = train_small["date_time"].dt.year
train_small["month"] = train_small["date_time"].dt.month
# test_small = pd.read_csv("data/test_small.csv")

In [12]:
# train_small_ = train_small.set_index("srch_id")
# test_small_ = test_small.set_index("srch_id")
train_small.sort_values("date_time")

,srch_id,date_time,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,...,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,click_bool,gross_bookings_usd,booking_bool,year,month
390360,26168,2012-11-01 00:35:47,32,220,219,113405,2,3.5,0,2.08,...,118.528297,44.042252,-1.000000,0.000000,26.000000,0,281.632385,0,2012,11
390346,26168,2012-11-01 00:35:47,32,220,219,65606,4,4.0,1,2.89,...,29.486151,-1.737388,0.000000,0.000000,-3.166452,0,314.792725,0,2012,11
390347,26168,2012-11-01 00:35:47,32,220,219,68914,2,3.0,1,2.20,...,18.356794,97.117226,0.000000,0.000000,-0.928227,0,532.156860,0,2012,11
390348,26168,2012-11-01 00:35:47,32,220,219,74474,3,4.5,1,2.40,...,-57.045246,-72.180244,26.443462,14.338161,27.148546,0,169.626953,0,2012,11
390349,26168,2012-11-01 00:35:47,32,220,219,81437,3,4.0,1,2.30,...,-7.126264,-12.799749,0.000000,0.000000,-29.721807,0,307.371124,0,2012,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413299,27674,2013-06-30 23:20:46,4,50,219,124778,4,4.0,1,4.29,...,-8.716877,26.564783,-2.329306,-5.041114,1.337687,0,190.946091,0,2013,6
413300,27674,2013-06-30 23:20:46,4,50,219,128155,4,4.5,1,2.20,...,-61.522110,99.727844,11.663659,-7.837674,-56.536709,0,593.529663,0,2013,6
413301,27674,2013-06-30 23:20:46,4,50,219,134288,4,4.5,1,3.87,...,13.934949,192.455200,11.891241,-0.453902,-30.295862,0,187.466888,0,2013,6
413291,27674,2013-06-30 23:20:46,4,50,219,87490,3,4.0,1,1.95,...,-37.766422,-75.318680,7.720039,-7.935442,-43.852844,0,501.542908,0,2013,6


In [9]:
train = train_small[((train_small.year == 2012) | ((train_small.year == 2013) & (train_small.month < 5)))]
test = train_small[((train_small.year == 2013) & (train_small.month >= 5))]

In [ ]:
# predict where booking_bool is 1